# Question 1

For the LP problem to have a finite solution, the region of constraints must be bounded, i.e., the space $\{x: Ax\leq b\}$ must be closed and bounded and not an empty set.  What condition must hold in order for there to be a finite solution to this problem? What is the optimal value of the objective function?

To obtain the optimal value of the objective function. We can derive the Lagrangian and apply KKT conditions. 

$$L = m^Tx+\lambda(Ax-b)$$

1. **Stationarity condition**:
$$
\mathbf{m} + \mathbf{A^T \lambda} = 0
$$

Where $\lambda$ is the vector of Lagrange multipliers for the inequality constraints. Since $A$ is invertible, we get $\lambda = -(A^T)^{-1}m$.

$$L = m^Tx+-(A^T)^{-1}m(Ax-b)$$

2. **Primal feasibility**:
$$
\mathbf{Ax} \leq \mathbf{b}
$$

3. **Dual feasibility**:
$$
\lambda \geq 0
$$

4. **Complementary slackness**:
$$
\lambda_i ( \mathbf{a_i^T x} - b_i ) = 0, \quad \text{for all } i
$$

Where $ \mathbf{a_i^T} $ is the $i$-th row of matrix $ \mathbf{A} $ and $ b_i $ is the $i$-th element of vector $\mathbf{b}$. This condition states that if the $i$-th constraint is not binding at the optimal solution, then the corresponding Lagrange multiplier $\lambda_i$ is zero.

Solved $x = A^{-1}b$. 

Under these conditions, the strong duality holds and the solution is optimal. The optimal value is $m^TA^{-1}b$

# Question 2

Solve the optimization problem below:

$$ \text{minimize} \quad x^2 + 1 $$

$$ \text{subject to} \quad (x-2)(x-4) \leq 0 $$

Feasible region: $x \in [2,4]$

The optimal value is $\min\{x^2+1\} = 5$

We have the Lagrangian 
$$L(\lambda)= x^2 + 1 + \lambda (x-2)(x-4)$$

And $$\frac{\partial L}{\partial x}=2x + 2\lambda x - 6\lambda = 0$$
So we have $$x = \frac{3\lambda}{1+\lambda}$$ 
Thus the dual problem is $$ \max \;\;\;(\frac{3\lambda}{1+\lambda})^2+1+\lambda(\frac{3\lambda}{1+\lambda}-2)(\frac{3\lambda}{1+\lambda}-4)$$

$$s.t. \lambda \geq 0$$

Using FOC we may compute the max value for the dual problem to be $5$ when $\lambda = 2$. Since $V_D = V_P$, we know that the strong duality holds.

# Question 3

\begin{align*}
\text{minimize} \quad & \frac{1}{2} x^T C x - \gamma \exp\left(- \sum_{i: x_i > 0} x_i \cdot \ln(x_i)\right) \\
\text{subject to} \quad & u^T x = 1 \\
& x \in \mathbb{R}^n
\end{align*}

The problem can be converted into a unconstrained problem as follows:

$$\text{minimize} \quad \frac{1}{2} x^T C x - \gamma \exp\left(- \sum_{i: x_i > 0} x_i \cdot \ln(x_i)\right)+ F(u^Tx-1)$$

where $F(0)= 0$ and $F(x)=\infty$ if $x\neq 0$.

In [37]:
import cvxpy as cp
import numpy as np

# Covariance matrix
C = np.array([[0.4961, 0.2414, 0.2141], 
              [0.2414, 0.3588, 0.1165], 
              [0.2141, 0.1165, 0.4057]])

# Define the optimization variable
x = cp.Variable(3)

# Objective function
objective = cp.Minimize(0.5 * cp.quad_form(x, C))

# Constraints
constraints = [
    cp.sum(x) == 1,  # Budget constraint
    x >= 0           # Long-only constraint
]

# Form and solve the optimization problem
prob = cp.Problem(objective, constraints)
prob.solve()

# This method is more stable than gradient descent!
print("Optimal portfolio weights:", x.value)

Optimal portfolio weights: [0.06798985 0.50363351 0.42837664]


### Conclusion 

Using a more stable optimization method, we verified the optimal portfolio weights as shown in (4.6#)

Below is a less accurate gradient descent method.

In [69]:
import numpy as np

def objective_function(x, C, gamma):
    quadratic_term = 0.5 * np.dot(x.T, np.dot(C, x))
    exp_term = -gamma * np.exp(-np.sum(x[x > 1e-9] * np.log(x[x > 1e-9])))
    return quadratic_term + exp_term

def compute_gradient(x, C, gamma):
    # Gradient for quadratic term
    grad_quad = np.dot(C, x)
    
    # Gradient for exponential term
    mask = x > 1e-9
    grad_exp_term = np.zeros_like(x)
    grad_exp_term[mask] = -gamma * np.exp(-np.sum(x[mask] * np.log(x[mask]))) * (np.log(x[mask]) + 1)
    
    return grad_quad + grad_exp_term

def gradient_descent(C, u, gamma, max_iters=1000, alpha=0.001, tol=1e-6):
    x = np.array([0.06, 0.5, 0.44])
    prev_obj = float('inf')
    
    for i in range(max_iters):
        gradient = compute_gradient(x, C, gamma)
        
        # Update step
        x -= alpha * gradient
        
        # Projection to ensure positivity
        x = np.maximum(x, 1e-9)
        
        # Projection to ensure u^Tx = 1
        scaling_factor = np.dot(u, x)
        x /= scaling_factor
        
        # Check for convergence
        current_obj = objective_function(x, C, gamma)
        if abs(prev_obj - current_obj) < tol:
            break
        prev_obj = current_obj

    return x

C = np.array([[0.4961, 0.2414, 0.2141], 
              [0.2414, 0.3588, 0.1165], 
              [0.2141, 0.1165, 0.4057]])
u = np.array([1, 1, 1])
gamma = 0

optimized_x = gradient_descent(C, u, gamma)
print(optimized_x)

[0.059599   0.50024938 0.44015163]


### Verify that you get an equal-weighted portfolio for a large $\lambda$.

In [88]:
import cvxpy as cp
import numpy as np

# Covariance matrix
C = np.array([[0.4961, 0.2414, 0.2141], 
              [0.2414, 0.3588, 0.1165], 
              [0.2141, 0.1165, 0.4057]])

# Define the optimization variable
x = cp.Variable(3, nonneg=True)

# Define a large lambda
lambda_large = 10000

# Objective function with entropy term
objective = cp.Minimize(0.5 * cp.quad_form(x, C) - lambda_large * cp.sum(cp.entr(x)))

# Constraints
constraints = [
    cp.sum(x) == 1  # Budget constraint
]

# Form and solve the optimization problem
prob = cp.Problem(objective, constraints)
prob.solve()

print("Optimal portfolio weights for large lambda:", x.value)


Optimal portfolio weights for large lambda: [0.33333175 0.33333409 0.33333416]


/Users/Eric/opt/anaconda3/envs/dsm/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


### Conclusion

We verified that the output is an equal-weighted portfolio.

In [92]:
import cvxpy as cp
import numpy as np

# Your covariance matrix from before:
C = np.array([[0.4961, 0.2414, 0.2141], 
              [0.2414, 0.3588, 0.1165], 
              [0.2141, 0.1165, 0.4057]])

# Expected returns for each asset (you would need to provide this)
# For this example, I'll assume a random expected return for each asset:
expected_returns = np.array([0.02, 0.03, 0.04]) 

# Define optimization variables
x = cp.Variable(3)
gamma = cp.Parameter(nonneg=True)  # gamma is a parameter in our problem

# Define the objective function: minimize (x^T * C * x - gamma * x^T * expected_returns)
objective = cp.Minimize(cp.quad_form(x, C) - gamma * x.T @ expected_returns)

# Define the constraints
constraints = [
    cp.sum(x) == 1,  # Sum of weights is 1
    x >= 0,          # No short-selling
    x[0] == 0.20     # Weight of CHF is 20%
]

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Since gamma is a parameter, we can compute the portfolio for different values of gamma
gamma_vals = np.logspace(-2, 3, 50)  # You can adjust this range
weights = []

for val in gamma_vals:
    gamma.value = val
    problem.solve()
    weights.append(x.value)

# Find the gamma value that gives 20% weight to CHF
for idx, weight in enumerate(weights):
    if abs(weight[0] - 0.20) < 1e-5:  # A small tolerance value
        optimal_gamma = gamma_vals[idx]
        break

print(f"Optimal gamma that gives 20% weight to CHF: {optimal_gamma}")
print(f"Weights for this gamma value: {weights[idx]}")


Optimal gamma that gives 20% weight to CHF: 0.01
Weights for this gamma value: [0.2        0.42492944 0.37507056]
